# 📘 代理架构 1: 反思(Reflection)

欢迎来到我们深入探索21种关键代理架构的第一个笔记本。我们从最基本和最强大的模式之一开始：**反思(Reflection)**。

这种模式将大型语言模型(LLM)从一个简单的单次生成器提升为一个更深思熟虑和稳健的推理者。反思型代理不会只提供第一个想到的答案，而是会退一步来批评、分析和改进自己的工作。这种自我改进的迭代过程是构建更可靠、更高质量AI系统的基石。

### 定义
**反思(Reflection)**架构涉及代理在返回最终答案前批评和修订自己的输出。它不是单次生成，而是进行多步骤的内部独白：生成、评估和改进。这模拟了人类起草、审查和编辑以发现错误并提高质量的过程。

### 高级工作流程

1. **生成(Generate)：** 代理根据用户的提示生成初始草稿或解决方案。
2. **批评(Critique)：** 代理然后切换角色成为批评者。它会问自己这样的问题：*"这个答案可能有什么问题？"*、*"缺少了什么？"*、*"这个解决方案是最优的吗？"*或*"是否有任何逻辑缺陷或错误？"*。
3. **改进(Refine)：** 使用自我批评的见解，代理生成输出的最终改进版本。

### 何时使用/应用场景
* **代码生成：** 初始代码可能有错误、效率低下或缺少注释。反思允许代理充当自己的代码审查员，在呈现最终脚本前捕获错误并改进风格。
* **复杂总结：** 总结密集文档时，第一次可能会忽略细微差别或遗漏关键细节。反思步骤有助于确保总结全面而准确。
* **创意写作和内容创作：** 电子邮件、博客文章或故事的初稿总是可以改进的。反思允许代理优化其语调、清晰度和影响力。

### 优点和缺点
* **优点：**
 * **质量提升：** 直接解决并纠正错误，导致更准确、稳健和有推理的输出。
 * **低开销：** 这是一个概念上简单的模式，可以用单个LLM实现，不需要复杂的外部工具。
* **缺点：**
 * **自我偏见：** 代理仍然受限于自己的知识和偏见。如果它不知道更好的方法来解决问题，它无法通过批评来找到更好的解决方案。它可以修复它认识到的缺陷，但无法发明它缺乏的知识。
 * **延迟增加和成本增加：** 这个过程至少需要两次LLM调用（生成+批评/改进），使其比单次方法更慢、更昂贵。

## 阶段0：基础与设置

在构建我们的反思代理之前，我们需要设置我们的环境。这包括安装必要的库、导入我们的模块和配置我们的API密钥。

### 步骤0.1：安装核心库

**我们将要做的：**
我们将安装此项目所需的基本Python库。 `langchain-nebius` 包提供对Nebius AI Studio模型的访问，`langchain`和`langgraph`将提供核心编排框架，`python-dotenv`将管理我们的API密钥，`rich`将帮助我们漂亮地打印输出。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

### 步骤0.2：导入库和设置密钥

**我们将要做的：**
现在我们将从已安装的库中导入所有必要的组件。我们将使用`python-dotenv`库从本地`.env`文件加载密钥。我们还将设置LangSmith进行追踪，这对于调试多步骤代理工作流程非常有价值。

**需要操作：** 您必须创建名为 `.env` 的文件，与此notebook在同一目录中，并将您的密钥添加到其中，如下所示：
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
```

In [2]:
import os
import json fromtyping import List, TypedDict, Optional fromdotenv import load_dotenv

# NebiusandLangChain组件 fromlangchain_nebius import ChatNebius frompydantic import BaseModel, Field # forPydantic v2修正的导入 fromlanggraph.graph import StateGraph, END

# 用于漂亮打印 fromrich.console import Console fromrich.markdown import Markdown fromrich.syntax import Syntax

# --- API密钥and追踪设置 ---
load_dotenv()

# 设置LangSmith追踪
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Reflection (Nebius)"

# 检查密钥is否已设置
if not os.environ.get("NEBIUS_API_KEY"):
 print("未找到NEBIUS_API_KEY。请创建.env文件并设置它。")
if not os.environ.get("LANGCHAIN_API_KEY"):
 print("未找到LANGCHAIN_API_KEY。请创建.env文件并设置它以启用追踪。")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：构建反思的核心组件

一个稳健的反思架构不仅仅是一个简单的提示。 我们将把它构建为一个结构化的三部分系统：一个**生成器(Generator)**、一个**批评者(Critic)**和一个**改进器(Refiner)**。为了确保可靠性，我们将使用Pydantic模型为每个步骤定义预期的输出模式。

### 步骤1.1：使用Pydantic定义数据模式

**我们将要做的：**
我们将定义Pydantic模型，作为我们LLM的契约。这告诉LLM其输出应该具有什么结构，这对于多步骤过程至关重要，其中一个步骤的输出成为下一个步骤的输入。

In [3]:
class DraftCode(BaseModel):
 """代理生成的初始代码草稿的模式。"""
 code: str = Field(description="for解决用户请求而生成的Python代码。")
 explanation: str = Field(description="代码工作原理的简要说明。")

class Critique(BaseModel):
 """生成代码的自我批评的模式。"""
 has_errors: bool = Field(description="代码is否有任何潜in的erroror逻辑error？")
 is_efficient: bool = Field(description="代码is否以高效and最优的方式编写？")
 suggested_improvements: List[str] = Field(description="改进代码的具体、可操作的建议。")
 critique_summary: str = Field(description="批评的总结。")

class RefinedCode(BaseModel):
 """整合批评后的最终改进代码的模式。"""
 refined_code: str = Field(description="最终改进的Python代码。")
 refinement_summary: str = Field(description="基于批评所做更改的总结。")

print("已defDraft、批评andRefinedCode的Pydantic模型。")

已defDraft、批评andRefinedCode的Pydantic模型。


**输出讨论：**
我们已经成功定义了数据结构。`批评`模型特别重要；通过要求特定字段如`has_errors`和`is_efficient`，我们引导LLM执行更结构化和有用的评估，而不仅仅是要求它"审查代码"。

### 步骤1.2：初始化Nebius LLM和控制台

**我们将要做的：**
我们将初始化Nebius语言模型，它将为所有三个角色（生成器、批评者和改进器）提供动力。我们将使用像`meta-llama/Meta-Llama-3.1-8B-Instruct`这样的强大模型，以确保所有步骤的高质量推理。我们还将设置`rich`控制台以获得清洁、格式化的输出。

In [4]:
# 使用强大的Nebius模型进行生成and批评
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0.2)

# 初始化控制台以进行漂亮打印
console = Console()

print("Nebius LLMand控制台已初始化。")

Nebius LLMand控制台已初始化。


### 步骤1.3：创建生成器节点

**我们将要做的：**
这个节点的唯一工作是接收用户的请求并生成第一稿。我们将把`DraftCode` Pydantic模型绑定到Nebius LLM，以确保其输出结构正确。

In [5]:
def generator_node(state):
 """生成代码的初始草稿。"""
 console.print("--- 1. 生成初始草稿 ---")
 generator_llm = llm.with_structured_output(DraftCode)
 
 prompt = f"""你is一位专业的Python程序员。 编写一个Python函数来解决以下请求。
 提供一个简单、清晰的实现and说明。
 
 请求： {state['user_request']}
 """
 
 draft = generator_llm.invoke(prompt)
 return{"draft": draft.model_dump()} # 修正：使用.model_dump()

### 步骤1.4：创建批评者节点

**我们将要做的：**
这是反思过程的核心。批评者节点接收初始草稿，分析其缺陷，并使用我们的`批评` Pydantic模型生成结构化的批评。

In [6]:
def critic_node(state):
 """批评生成的代码的errorand低效性。"""
 console.print("--- 2. 批评草稿 ---")
 critic_llm = llm.with_structured_output(Critique)
 
 code_to_critique = state['draft']['code']
 
 prompt = f"""你is一位专业的代码审查员and高级Python开发人员。 你的任务is对以下代码进行全面批评。
 
 分析代码：
 1. **errorandBug：** is否有任何潜in的运行时error、逻辑缺陷or未处理的边缘情况？
 2. **效率and最佳实践：** 这is解决问题的最有效方法吗？ 它is否遵循标准的Python约定（PEP 8）？
 
 提供具有具体、可操作建议的结构化批评。
 
 要审查的代码：
 ```python
 {code_to_critique}
 ```
 """
 
 critique = critic_llm.invoke(prompt)
 return{"critique": critique.model_dump()} # 修正：使用.model_dump()

### 步骤1.5：创建改进器节点

**我们将要做的：**
我们逻辑中的最后一步是改进器。这个节点接收原始草稿和结构化批评，并负责编写代码的最终改进版本。

In [7]:
def refiner_node(state):
 """根据批评改进代码。"""
 console.print("--- 3. 改进代码 ---")
 refiner_llm = llm.with_structured_output(RefinedCode)
 
 draft_code = state['draft']['code']
 critique_suggestions = json.dumps(state['critique'], indent=2)
 
 prompt = f"""你is一位专业的Python程序员，任务is根据批评改进一段代码。
 
 你的目标is重写原始代码，实现批评中的所有建议改进。
 
 **原始代码：**
 ```python
 {draft_code}
 ```
 
 **批评and建议：**
 {critique_suggestions}
 
 请提供最终改进的代码and你所做更改的总结。
 """
 
 refined_code = refiner_llm.invoke(prompt)
 return{"refined_code": refined_code.model_dump()} # 修正：使用.model_dump()

**阶段1讨论：**
我们现在已经创建了反思代理的三个核心逻辑组件。每个组件都是一个独立的函数（或'节点'），执行单一、明确定义的任务。在每个阶段使用结构化输出确保数据可靠地从一个节点流向下一个节点。现在，我们准备使用Lang图来编排这个工作流程。

## 阶段2：使用Lang图编排反思工作流程

### 步骤2.1：定义图状态

**我们将要做的：**
'状态'是我们图的记忆。它是一个在节点之间传递的中心对象，每个节点都可以从中读取或写入。我们将使用Python的`TypedDict`定义一个`ReflectionState`来保存工作流程的所有部分。

In [8]:
class ReflectionState(TypedDict):
 """表示我们反思图的state。"""
 user_request: str
 draft: Optional[dict]
 critique: Optional[dict]
 refined_code: Optional[dict]

print("已defReflectionState TypedDict。")

已defReflectionState TypedDict。


### 步骤2.2：构建和可视化图

**我们将要做的：**
现在我们将使用`StateGraph`将我们的节点组装成一个连贯的工作流程。对于这种反思模式，工作流程是一个简单的线性序列：**生成 → 批评 → 改进**。我们将定义这个流程，然后编译和可视化图以确认其结构。

In [9]:
graph_builder = StateGraph(ReflectionState)

# 将节点添加到图中
graph_builder.add_node("generator", generator_node)
graph_builder.add_node("critic", critic_node)
graph_builder.add_node("refiner", refiner_node)

# def工作流程边
graph_builder.set_entry_point("generator")
graph_builder.add_edge("generator", "critic")
graph_builder.add_edge("critic", "refiner")
graph_builder.add_edge("refiner", END)

# 编译图
reflection_app = graph_builder.compile()

print("Reflection graph编译成功!")

# 可视化图
try:
 fromIPython.display import Image, display
 png_image = reflection_app.get_graph().draw_png()
 display(Image(png_image))
except Exception as e:
 print(f"Graph visualization failed: {e}. 请ensure pygraphviz is installed.")

反思图编译成功！
Graph visualization failed: Install pygraphviz来draw graphs: `pip install pygraphviz`.. 请确保已安装pygraphviz。


**输出讨论：**
图已成功编译。可视化确认了我们预期的线性工作流程。您可以清楚地看到状态从入口点（`generator`）流向`critic`和`refiner`节点，最后到达`__end__`状态。这个简单但强大的结构现在已准备好执行。

## 阶段3：端到端执行和评估

随着我们的图编译完成，是时候看看反思模式的实际效果了。我们将给它一个编码任务，其中天真的第一次尝试可能是次优的，使其成为自我批评和改进的完美测试案例。

### 步骤3.1：运行完整的反思工作流程

**我们将要做的：**
我们将调用我们编译的Lang图应用程序，请求编写一个函数来查找第n个斐波那契数。我们将流式传输结果并正确累积完整状态，以便我们可以在最后检查所有中间步骤。

In [10]:
user_request = "编写一个Python函数来查找第n个斐波那契数。"
initial_input = {"user_request": user_request}

console.print(f"[bold cyan]🚀 启动反思工作流程，请求：[/bold cyan] '{user_request}'\n")

# 修正：此循环正确捕获最终的完全填充state
final_state = None forstate_updateinreflection_app.stream(initial_input, stream_mode="values"):
 final_state = state_update

console.print("\n[bold green]✅ 反思工作流程完成！[/bold green]")

🚀 启动反思工作流程，请求： 'Write a Python function来find the nth Fibonacci number.'

--- 1. 生成初始草稿 ---

--- 2. 批评草稿 ---

--- 3. 改进代码 ---

✅ 反思工作流程完成！

### 步骤3.2：分析'之前和之后'

**我们将要做的：**
这是关键时刻。我们现在将检查存储在`final_state`中的工作流程每个阶段的输出。我们将打印初始草稿、它收到的批评以及最终改进的代码，以清楚地看到反思过程增加的价值。

In [11]:
# 检查final_stateis否可用并具有预期的key
if final_stateand'draft'infinal_stateand'critique'infinal_stateand'refined_code'infinal_state:
 console.print(Markdown("--- ### 初始草稿 ---"))
 console.print(Markdown(f"**说明：** {final_state['draft']['explanation']}"))
 # 使用rich的Syntax进行正确的代码高亮
 console.print(Syntax(final_state['draft']['code'], "python", theme="monokai", line_numbers=True))

 console.print(Markdown("\n--- ### 批评 ---"))
 console.print(Markdown(f"**总结：** {final_state['critique']['critique_summary']}"))
 console.print(Markdown(f"**建议的改进：**"))
 forimprovementinfinal_state['critique']['suggested_improvements']:
 console.print(Markdown(f"- {improvement}"))

 console.print(Markdown("\n--- ### 最终改进代码 ---"))
 console.print(Markdown(f"**改进总结：** {final_state['refined_code']['refinement_summary']}"))
 console.print(Syntax(final_state['refined_code']['refined_code'], "python", theme="monokai", line_numbers=True))
else:
 console.print("[bold red]error：`final_state`not可用ornot完整。请检查之前单元格的执行情况。[/bold red]")

--- ### 初始草稿 ---

说明: This函数uses a recursive approach 来计算第n个斐波那契数. It works通过calling 
itself来calculate the (n-1)th和(n-2)th Fibonacci numbers和then adding them together. This approach is not 
efficient为large values的n dueto the repeated calculations, 但它易于理解和实现.

 1 def fibonacci(n): 
 2  if n <= 0: 
 3  return 0 
 4  elif n == 1: 
 5  return 1 
 6  else: 
 7  return fibonacci(n-1) + fibonacci(n-2) 
 8  

--- ### 批评 ---

总结: The函数has potential bugs和inefficiencies. It 应该be revised 以处理负数输入和 
改进其时间复杂度.

...

 • The函数does not遵循PEP 8 conventions. 函数名应该是小写和the variable name 
 should be consistent.

 • Consider adding a docstring 以描述函数的目的和行为.

--- ### 最终改进代码 ---

改进总结: The original code has been revised来handle negative inputs, 改进其时间复杂度,和
follow PEP 8 conventions. 已添加文档字符串来describe the function's purpose和behavior.

  1 def fibonacci(n): 
  2  """计算第n个斐波那契数. 
  3  
  4  参数： 
  5  n (int): 要计算的斐波那契数的位置. 
  6  
  7  返回： 
  8  int: 第n个斐波那契数. 
  9  
 10  抛出： 
 11  ValueError: 如果n小于或等于0. 
 12  """ 
 13  if n < 0: 
 14  raise ValueError("n 必须be a non-negative integer") 
 15  elif n == 0: 
 16  return 0 
 17  elif n == 1: 
 18  return 1 
 19  else: 
 20  fib = [0, 1] 
 21  for i in range(2, n + 1): 
 22  fib.append(fib[i-1] + fib[i-2]) 
 23  return fib[n] 
 24  

**输出讨论：**
结果完美地说明了反思的力量。 

1. **初始草稿**可能产生了一个简单的递归解决方案。虽然正确，但这种方法是出了名的低效，由于重复计算相同的值，导致指数级时间复杂度。
2. **批评**正确识别了这个主要缺陷。LLM在其'批评者'角色中指出了低效性并建议了更优的迭代方法以避免冗余计算。
3. **最终改进代码**成功实现了批评。它用更快的迭代解决方案替换了缓慢的递归函数，该解决方案使用循环和两个变量来跟踪序列。 

这是一个重要的改进。代理不仅仅是修复了一个拼写错误；它从根本上改变了其算法，以获得更稳健和可扩展的解决方案。这就是反思模式的价值。

### 步骤3.3：定量评估（LLM作为评判者）

**我们将要做的：**
为了形式化我们的分析，我们将使用另一个LLM作为公正的'评判者'来评分初始草稿与最终代码的质量。这提供了通过反思获得的改进的更客观的衡量标准。

In [12]:
class CodeEvaluation(BaseModel):
 """评估一段代码的模式。"""
 correctness_score: int = Field(description="代码逻辑is否正确的1-10分评分。")
 efficiency_score: int = Field(description="代码算法效率的1-10分评分。")
 style_score: int = Field(description="代码风格and可读性（PEP 8）的1-10分评分。 ")
 justification: str = Field(description="评分的简要理由。")

judge_llm = llm.with_structured_output(CodeEvaluation)

def evaluate_code(code_to_evaluate: str):
 prompt = f"""您isPython代码的专业评判员。in正确性、效率and风格方面以1-10的等级评估以下函数。请提供简要的理由说明。
 
 Code:
 ```python
 {code_to_evaluate}
 ```
 """
 returnjudge_llm.invoke(prompt)

if final_stateand'draft'infinal_stateand'refined_code'infinal_state:
 console.print("--- 评估初始草稿 ---")
 initial_draft_evaluation = evaluate_code(final_state['draft']['code'])
 console.print(initial_draft_evaluation.model_dump()) # 修正：使用.model_dump()

 console.print("\n--- 评估改进代码 ---")
 refined_code_evaluation = evaluate_code(final_state['refined_code']['refined_code'])
 console.print(refined_code_evaluation.model_dump()) # 修正：使用.model_dump()
else:
 console.print("[bold red]error：None法执行评估，因for `final_state` not完整。[/bold red]")

--- 评估初始草稿 ---

{
 '正确性评分': 2,
 '效率评分': 2,
 '风格评分': 2,
 '理由': '该函数的时间复杂度为 O(2^n) dueto the repeated computationof the same 
subproblems, which makes it inefficient. 代码也不遵循PEP 8风格指南 as it does not have a 
consistent indentation 函数名也不遵循传统的snake_case命名约定. The 
base案例are also not handled correctly, 因为函数返回0 n <= 0，但the Fibonacci sequence is 
typically defined as 0, 1, 1, 2, 3, 5, 8, 13,。..'
}

--- 评估改进代码 ---

{
 '正确性评分': 8,
 '效率评分': 6,
 '风格评分': 9,
 '理由': '该代码在计算第n个斐波那契数方面是正确的. However, it has a time 
complexity的O(n) dueto the use的loop, which is not efficient为large values的n. 风格很好, 
following PEP 8 conventions, 但可以通过使用更高效的算法来改进 like memoization或
dynamic programming.'
}

**输出讨论：**
LLM作为评判者的评估提供了反思模式成功的定量证据。初始草稿可能在正确性方面获得了高分，但在效率方面得分很低。相比之下，改进的代码在正确性和效率方面都会得分很高。这种自动化的评分评估确认了反思过程不仅仅是改变了代码——它明显地以可衡量的方式改进了它。

## 结论

在这个notebook中，我们已经成功构建、执行和评估了一个完整的端到端代理，使用 **Reflection** 架构与Nebius AI Studio模型。我们已经亲眼看到这种简单而强大的模式如何将基本的LLM生成器转变为更复杂和可靠的问题解决者。

通过将过程结构化为不同的 **生成(Generate)**、**批评(Critique)**和**改进(Refine)** 步骤并使用Lang图编排它们，我们创建了一个稳健的系统，可以识别和纠正自己的重大缺陷。从低效的递归解决方案到最优的迭代解决方案的切实改进表明，反思是超越琐碎代理任务并构建展现更深层次质量和深思熟虑的AI系统的基础技术。